In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dataset-age:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4784648%2F8101805%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240427%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240427T165144Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6866ea94b91b07a6aeaa2f2271f714acd8f98cdfb08c83c2f023174c3e8749c735f52c4e526f4b6e57aeb7755577dd46870504762dcbc5f999173a152b3795300cdaa44d79f02eca7776a16f476a3a27ef5e1ea92ce86e91477099da41dbf3e4c39ef0eb7addff516a3ddb0aa0a0d4bf0358d5a7a86584a8ec22f69123439cf2350d113e2f4d50fc0365766391e58cc2653a08edbdf9265ce24dd33fead9be1036d3acaf230ce5659a24dc08f6d768b8adb5a30619111274ec7d70a186243e4d65d7f0d1e95d2098910a86f42b57fc2be8291964f7c383f612c01b4ceba4a594a5d9d333e0bdb27005fa0a5e21895f4609437933d529f751a69500680be20d25,age-metadata:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4784789%2F8102004%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240427%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240427T165144Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4a832f5027222eb636d3758837fa9cbdf6c8606993ec94ac0edf8cbba7e9f6b3ae01bfcca8c96f3f56723164b9e935094eb0df9f9f15a5f818c2d741ae5a92e62b4a1417c2fb96b0cfe53b6fcf6954742c13df06f6e8c9935baa75be1dd2b079b425d1c4c5a6efb6fc5c99a9a0842f04e545079aafa37c4cd9b25905e862a5d660e4e57ff55c169eb40624f52b2cd6b8388bdddb866c0f294fd43281990a630594d37f72cefefb043864100fe152da3d87f43a3b45693e81f0dd0290388c54e2d0dafb7590b8af5724f9209a06fa7b7e64c47271cd4189b1851590fa6943d31baaf66369d5b1cd99aa51e83604a694a1f213ff5996e2e4c6846986e697403d1b,age-testdata:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4865651%2F8210545%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240427%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240427T165144Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4bcc568fbe7032655e505ff10d0dd2c334799cc39ba3aa27893a559756deed14a8a7cdc888db4e7bc7a5f9f48cb1e7475297853a688e9cb9955d2a08e3ab7f1dbea42546f19d5c7c153ca5c37d5495f25b9a6962d93b9ee7844d4e7ee4492e3e7cf399f2027c43bcca7d9de886b74d86918d00022345678bd69d7bee5b78919b6ab868883243ba51fc51548c9ddc425fcabad0d7a76b66b6ecd5b0ebad4b4ac5fdfd79e8d775de86be256d4fc7145712dfc4cb40107b65596728109784a1eaf45bce7129d681c025444ccd8b6edd05ff4c66e838bd5dad36fa10cd76775190789d8af5a7b05de925f1fef6b682f8b539a24782e73fb897380755a41d2739dc66,age-testimages:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4865659%2F8210555%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240427%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240427T165144Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D76f5edd8b1fc92ce02b6c3d5ed138e74d5be5d9c8a8e3b1f3af4b9cff6b551bcc5b964a0a87ba6f8a59c6e7dc46cb77f9467e64c4025f68588b933fdfa693e3535e2685e5c3fd652b39cd75a81e635a359f4cce9a53da4ce329c36fd8ae079e51dfbd9a6275bc0e52fea2817872000e54f9c97b535390df0d587147189c49a7b812a6cb63b3e2b054f14ddec02907bd86b6e19061977e370c15d13f2d8400fa1100b8baa4324d5264f1506796c22b923b2b316f139a618f0f7e0d2a4c0d933d3923b11ae17d6f5b79d0b3a48a50dd8646bf8b160766677525deed3fd80ebe9e72bb7bef969a5eb74a4fd1532b4f6533976a8c8c4b189530b9bbf89d5207fc060'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 319179331 bytes downloaded
Downloaded and uncompressed: dataset-age
[==================================================] 3804893 bytes downloaded
Downloaded and uncompressed: age-metadata
[==================================================] 23056 bytes downloaded
Downloaded and uncompressed: age-testdata
[==================================================] 9144495 bytes downloaded
Downloaded and uncompressed: age-testimages
Data source import complete.


In [2]:
import cv2
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input,InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.losses import LogCosh


In [3]:
def load_and_filter_images(image_paths, base_directory):
    valid_images = []
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    for img_path in image_paths:
        full_path = os.path.join(base_directory, img_path)  # Correctly join the base directory with the image path
        image = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            continue
        faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)
        if len(faces) == 1:
            valid_images.append(img_path)
    return valid_images

In [4]:
data = pd.read_csv('/kaggle/input/age-metadata/wiki_labels.csv')
data['full_path'] = data['full_path'].str.extract(r"(\d+\/[^']+\.png)")  # Regex to extract the relative path


In [5]:
base_directory = '/kaggle/input/dataset-age/wiki_labeled/wiki_labeled/'

In [6]:
valid_image_paths = load_and_filter_images(data['full_path'].dropna().tolist(), base_directory)
data = data[data['full_path'].isin(valid_image_paths)]

In [7]:
data['face_score'] = pd.to_numeric(data['face_score'], errors='coerce')
data = data[data['face_score'].notna() & (data['face_score'] != float('inf')) & data['second_face_score'].isna()]

In [8]:
base_year = 1.0
data['approx_dob_year'] = base_year + (data['dob'] - 366) / 365.25
data['age'] = data['photo_taken'] - data['approx_dob_year']
data['age'] = data['age'].round().astype('float64')
data = data[(data['age'] > 0) & (data['age'] <= 100)]


In [9]:
data['full_path'] = data['full_path'].str.extract(r"(\d+\/[^']+\.png)")

In [10]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.5, 1.5],
    validation_split=0.2
)


In [11]:
train_generator = datagen.flow_from_dataframe(
    dataframe=data,
    directory=base_directory,
    x_col='full_path',
    y_col='age',
    target_size=(100, 100),
    batch_size=32,
    color_mode='grayscale',
    class_mode='raw',
    subset='training'
)

Found 22836 validated image filenames.


In [12]:
validation_generator = datagen.flow_from_dataframe(
    dataframe=data,
    directory=base_directory,
    x_col='full_path',
    y_col='age',
    target_size=(100, 100),
    batch_size=32,
    color_mode='grayscale',
    class_mode='raw',
    subset='validation'
)


Found 5709 validated image filenames.


In [14]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import MeanAbsoluteError
model = Sequential([
    InputLayer(input_shape=(100, 100, 1)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(1024, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1)
])

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=Adam(learning_rate=0.0001), loss=LogCosh(),  metrics=[MeanAbsoluteError()])

In [15]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/50
713/713 [==============================] - 57s 69ms/step - loss: 15.8738 - mean_absolute_error: 13.8371 - val_loss: 13.2812 - val_mean_absolute_error: 13.3081 - lr: 1.0000e-04
Epoch 2/50
713/713 [==============================] - 48s 68ms/step - loss: 12.6144 - mean_absolute_error: 12.8670 - val_loss: 12.0610 - val_mean_absolute_error: 12.4289 - lr: 1.0000e-04
Epoch 3/50
713/713 [==============================] - 48s 68ms/step - loss: 12.2035 - mean_absolute_error: 12.6037 - val_loss: 12.4248 - val_mean_absolute_error: 12.8372 - lr: 1.0000e-04
Epoch 4/50
713/713 [==============================] - 48s 68ms/step - loss: 12.1448 - mean_absolute_error: 12.5569 - val_loss: 11.9432 - val_mean_absolute_error: 12.3551 - lr: 1.0000e-04
Epoch 5/50
713/713 [==============================] - 48s 68ms/step - loss: 11.9228 - mean_absolute_error: 12.3316 - val_loss: 11.4637 - val_mean_absolute_error: 11.8795 - lr: 1.0000e-04
Epoch 6/50
713/713 [==============================] - 48s 68ms/st

In [16]:

judge_datagen = ImageDataGenerator(rescale=1./255)

judge_data = pd.read_csv('/kaggle/input/age-testdata/wiki_judge.csv')




In [17]:
judge_data['full_path'] = judge_data['full_path'].str.extract(r"(\d+\.png)")

In [18]:
judge_generator = judge_datagen.flow_from_dataframe(
    dataframe=judge_data,
    directory='/kaggle/input/age-testimages/wiki_judge_images/wiki_judge_images/',
    x_col='full_path',
    y_col=None,
    target_size=(100, 100),
    batch_size=32,
    color_mode='grayscale',
    class_mode=None,
    shuffle=False
)

Found 1409 validated image filenames.


In [19]:
import numpy as np
predicted_ages = model.predict(judge_generator)
predicted_ages = predicted_ages.flatten()

predicted_ages = np.clip(predicted_ages, 0, 100)




45/45 [==============================] - 1s 19ms/step


In [20]:
submission_df = pd.DataFrame({
    'ID': judge_data['ID'],
    'age': predicted_ages
})

submission_df.to_csv('/content/sample_data/submissionLC.csv', index=False)
